In [1]:
import pandas as pd
import numpy as np

In [2]:
def expand_to_daily_long(df):
    all_results = []

    for sid, df_sid in df.groupby('stock_id'):

        # 日頻率 index
        full_dates = pd.date_range(
            df_sid['date'].min(),
            df_sid['date'].max(),
            freq='D'
        )
        temp = pd.DataFrame({'date': full_dates})
        temp['stock_id'] = sid

        # merge 季資料
        temp = temp.merge(df_sid, on=['date', 'stock_id'], how='left')

        # 只補財報欄位
        value_cols = [c for c in df.columns if c not in ['date','stock_id','type']]
        temp[value_cols] = temp[value_cols].ffill()

        all_results.append(temp)

    return pd.concat(all_results, ignore_index=True)

In [4]:
# 讀取股票資料
df = pd.read_csv("stock_data.csv")
# 讀取財報資料
df_bs = pd.read_pickle('BalanceSheet.pkl')
df_is = pd.read_pickle('IncomeStatement.pkl')
df_cf = pd.read_pickle('CashFlowsStatement.pkl')

In [5]:
# 欄位篩選
bs_needed_columns = [
    # 資產 Asset 
    '流動資產合計',
    '現金及約當現金',
    '應收帳款淨額',
    '存貨',
    '其他流動資產',
    '非流動資產合計',
    '不動產、廠房及設備',
    '無形資產',
    '使用權資產',
    '資產總計',   # 或 '資產總額'

    # 負債 Liability 
    '流動負債合計',
    '短期借款',
    '長期借款',
    '應付帳款',
    '其他流動負債',
    '非流動負債合計',
    '負債總額',

    # 權益 Equity 
    '普通股股本',
    '資本公積合計',
    '保留盈餘合計',
    '未分配盈餘',
    '權益總計',
    '歸屬於母公司業主之權益合計',
    '預收股款（權益項下）之約當發行股數',
    '母公司暨子公司所持有之母公司庫藏股股數'
]

is_clean_columns = [
    '營業收入',
    '營業成本',
    '營業毛利（毛損）淨額',
    '營業利益（損失）',          # EBIT
    '營業費用',
    '營業外收入及支出',
    '稅前淨利(淨損)',
    '所得稅費用（利益）',
    '本期淨利(淨損)',
    '基本每股盈餘（元）',
    '本期綜合損益總額',
    '其他收益及費損淨額',
    '其他綜合損益（淨額）',
    '淨利（淨損）歸屬於母公司業主',
    '淨利（淨損）歸屬於非控制權益',
    '綜合損益總額歸屬於非控制權益',
    '繼續營業單位本期淨利（淨損）',
    '已實現銷貨（損）益',
    '未實現銷貨（損）益'
]

cf_clean_columns = [
    # CFO（營業活動現金流） 
    '營業活動之淨現金流入（流出）',     
    '營運產生之現金流入（流出）',
    '本期稅前淨利（淨損）',
    '存貨（增加）減少',
    '應付帳款增加(減少)',
    '應付帳款-關係人增加(減少)',
    '應收帳款（增加）減少',
    '其他流動資產(增加)減少',
    '攤銷費用',
    '折舊費用',
    '利息收入',
    '利息費用',
    '支付之利息',
    '已實現銷貨損失（利益）',
    '未實現銷貨利益（損失）',
    '收益費損項目合計',

    # CFI（投資活動） 
    '投資活動之淨現金流入（流出）',
    '取得不動產、廠房及設備',
    '其他投資活動',
    '存出保證金減少',

    # CFF（籌資活動） 
    '籌資活動之淨現金流入（流出）',
    '償還長期借款',
    '償還公司債',
    '舉借長期借款',
    '租賃本金償還',
    '短期借款減少',

    # 現金餘額項目 
    '本期現金及約當現金增加（減少）數',
    '期初現金及約當現金餘額',
    '期末現金及約當現金餘額'
]

# 篩選出所出的中文欄位
data_bs_needed = df_bs[df_bs['origin_name'].isin(bs_needed_columns)]
data_is_needed = df_is[df_is['origin_name'].isin(is_clean_columns)]
data_cf_needed = df_cf[df_cf['origin_name'].isin(cf_clean_columns)]

In [7]:
# 轉換為 long format
df_bs_wide = data_bs_needed.pivot_table(
    index=['date', 'stock_id'],
    columns='type',
    values='value',
    aggfunc='first'
).reset_index()

df_is_wide = data_is_needed.pivot_table(
    index=['date', 'stock_id'],
    columns='type',
    values='value',
    aggfunc='first'
).reset_index()

df_cf_wide = data_cf_needed.pivot_table(
    index=['date', 'stock_id'],
    columns='type',
    values='value',
    aggfunc='first'
).reset_index()

# 資料排序
df_bs_wide.sort_values(by=['stock_id', 'date'], inplace=True)
df_is_wide.sort_values(by=['stock_id', 'date'], inplace=True)
df_cf_wide.sort_values(by=['stock_id', 'date'], inplace=True)

# date 轉 datetime
df_bs_wide['date'] = pd.to_datetime(df_bs_wide['date'])
df_is_wide['date'] = pd.to_datetime(df_is_wide['date'])
df_cf_wide['date'] = pd.to_datetime(df_cf_wide['date'])

In [8]:
for df in [df_bs_wide, df_is_wide, df_cf_wide]:
    df['stock_id'] = df['stock_id'].astype(str).str.strip()

common_ids = set(df_bs_wide['stock_id']) \
    & set(df_is_wide['stock_id']) \
    & set(df_cf_wide['stock_id']) 

common_ids = list(common_ids)
print("Common stock IDs across all datasets:", len(common_ids))

Common stock IDs across all datasets: 1006


In [9]:
df_bs_wide   = df_bs_wide[df_bs_wide['stock_id'].isin(common_ids)]
df_is_wide   = df_is_wide[df_is_wide['stock_id'].isin(common_ids)]
df_cf_wide   = df_cf_wide[df_cf_wide['stock_id'].isin(common_ids)]

df_bs_wide = df_bs_wide.sort_values(by=['stock_id', 'date']).reset_index(drop=True)
df_is_wide = df_is_wide.sort_values(by=['stock_id', 'date']).reset_index(drop=True)
df_cf_wide = df_cf_wide.sort_values(by=['stock_id', 'date']).reset_index(drop=True)

# 合併
df_merged = (
    df_bs_wide
    .merge(df_is_wide, on=['stock_id', 'date'], how='inner')
    .merge(df_cf_wide, on=['stock_id', 'date'], how='inner')
)

df_merged = df_merged.sort_values(['stock_id', 'date']).reset_index(drop=True)

In [10]:
k = df_merged[['stock_id', 'date']].drop_duplicates().shape[0]
print("季資料總列數：", len(df_merged), "唯一(stock_id,date)：", k)

季資料總列數： 40582 唯一(stock_id,date)： 40582


In [11]:
df_merged.to_csv("financial_data.csv", index=False)